In [23]:
import logging
import configparser
import pandas as pd
import pandas_ta as ta
import robin_stocks as rstocks
from robin_stocks import robinhood as r
from datetime import datetime
from pytz import timezone
import alive_progress
import traceback
from alive_progress import alive_bar
from icecream import ic
import asyncio
import sys
import time
import os
import random
from ratelimit import limits, sleep_and_retry
verbose_mode = False # Set to True to see more logging output
from tqdm import tqdm
from colorama import Fore, Back, Style


In [24]:
# The simplest trading strategy
Strategy = {
    'name': 'basic',

}

In [25]:
@sleep_and_retry
def get_last_100_days(coin):
    """
    The get_last_100_days function gets the last 100 days of a particular coin's data.
    :param coin: The coin to get data for
    :return: A DataFrame with the last 100 days of coin data
    :doc-author: Trelent
    """
    try:
        df = pd.DataFrame(r.crypto.get_crypto_historicals(coin, interval='hour', span='3month', bounds='24_7'))
        df = df.set_index('begins_at')
        df.index = pd.to_datetime(df.index)
        df = df.loc[:, ['close_price', 'open_price', 'high_price', 'low_price']]
        df = df.rename(columns={'close_price': 'close', 'open_price': 'open', 'high_price': 'high', 'low_price': 'low'})
        df = df.apply(pd.to_numeric)
        #note: could add coin name to df here too
        return df
    except Exception as e:
        print(f'Unable to get data for {coin}... {e}')
        return pd.DataFrame()

"""
Trading Strategy
- Trailing stop loss of 5% (sell if price drops 5% from peak)
- Buy if price increases 5% from trough
- Sell if price drops 2% from peak
- If the MACD Signal Line crosses above the MACD Line and an RSI of 30 or less is reached, buy the coin
- If the MACD Signal Line crosses below the MACD Line and an RSI of 70 or more is reached, sell the coin
- If the MACD Signal Line crosses below the MACD Line and an RSI of 70 or more is reached, sell the coin
- If the MACD Signal Line crosses above the MACD Line and an RSI of 30 or less is reached, buy the coin
"""

'\nTrading Strategy\n- Trailing stop loss of 5% (sell if price drops 5% from peak)\n- Buy if price increases 5% from trough\n- Sell if price drops 2% from peak\n- If the MACD Signal Line crosses above the MACD Line and an RSI of 30 or less is reached, buy the coin\n- If the MACD Signal Line crosses below the MACD Line and an RSI of 70 or more is reached, sell the coin\n- If the MACD Signal Line crosses below the MACD Line and an RSI of 70 or more is reached, sell the coin\n- If the MACD Signal Line crosses above the MACD Line and an RSI of 30 or less is reached, buy the coin\n'

In [26]:
def ordering_function(coin, spend_amount,side, df):
    """
    The ordering_function function takes in a coin, spend_amount, side and dataframe as arguments.
    The function then determines the side of the order (buy or sell) and places an order for that amount.
    If we are selling then we need to sell ALL of the coin that we have.
    
    :param coin: Determine which coin to buy or sell
    :param spend_amount: Determine how much money we want to spend on the order
    :param side: Determine whether to buy or sell
    :param df: Pass in the dataframe that is created by the get_data function
    :return: A dictionary of the order information
    :doc-author: Trelent
    """
    try:    
        # determine the side of the order
        # side = 'buy' or 'sell'
        amount_in = 'dollars' if side == 'buy' else 'amount'
        quantity_or_price = str(spend_amount / df['close'].iloc[-1]) if side == 'buy' else str(df['quantity_available'].iloc[-1])
        # place the buy order
        r.orders.order_crypto(
            symbol=coin,
            quantityOrPrice=float(quantity_or_price),
            amountIn=str(amount_in),
            side=side,
            timeInForce='gtc',
            jsonify=True
            )
        
        if side == 'buy': 
            print(f'Buying {quantity_or_price} {coin} at {df["close"].iloc[-1]}')
        else:
            print(f'Selling {quantity_or_price} {coin} at {df["close"].iloc[-1]}')
    except Exception as e:
        print(e)

def trading_function():
    coins = ['BTC', 'ETH', 'XRP', 'LTC', 'BCH', 'EOS', 'BNB', 'BSV', 'XLM', 'ADA']
    for coin in coins:
        df = pd.DataFrame(r.crypto.get_crypto_historicals(coin, interval='hour', span='week', bounds='24_7', info=None, jsonify=True))
        df['close'] = df['close_price'].astype(float)
        df['open'] = df['open_price'].astype(float)
        df['high'] = df['high_price'].astype(float)
        df['low'] = df['low_price'].astype(float)
    
        # get the MACD Signal Line from pandas_ta library
        macd_data = ta.macd(df['close'])
        df['macd_signal_line'] = macd_data['MACDh_12_26_9']
        df['macd_signal_line'] = df['macd_signal_line'].astype(float)
        # get the MACD Histogram from pandas_ta library
        df['macd_histogram'] = macd_data['MACDs_12_26_9']
        df['macd_histogram'] = df['macd_histogram'].astype(float)
        # get the MACD Line from pandas_ta library
        df['macd_line'] = macd_data['MACD_12_26_9']
        df['macd_line'] = df['macd_line'].astype(float)
        # get the RSI from pandas_ta library
        df['rsi'] = ta.rsi(df['close'])
        df['rsi'] = df['rsi'].astype(float)
        # get the atr from pandas_ta library
        df['atr'] = ta.atr(df['high'], df['low'], df['close'])
        df['atr'] = df['atr'].astype(float)
        # get the bollinger bands from pandas_ta library
        bb_data = ta.bbands(df['close'])
        df['bb_upperband'] = bb_data['BBU_5_2.0']
        df['bb_upperband'] = df['bb_upperband'].astype(float)
        df['bb_middleband'] = bb_data['BBM_5_2.0']
        df['bb_middleband'] = df['bb_middleband'].astype(float)
        df['bb_lowerband'] = bb_data['BBL_5_2.0']
        df['bb_lowerband'] = df['bb_lowerband'].astype(float)

        # get the last row of the dataframe
        last_row = df.iloc[-1]
    
        # make a decision based on the last row of the dataframe
        if last_row['macd_signal_line'] > 0 and last_row['macd_histogram'] > 0 and last_row['macd_line'] > 0 and last_row['rsi'] > 50 and last_row['close'] > last_row['bb_middleband'] and last_row['close'] > last_row['bb_upperband']:
            ordering_function(coin, 10, 'buy', df)
        elif last_row['macd_signal_line'] < 0 and last_row['macd_histogram'] < 0 and last_row['macd_line'] < 0 and last_row['rsi'] < 50 and last_row['close'] < last_row['bb_middleband'] and last_row['close'] < last_row['bb_lowerband']:
            ordering_function(coin, 10, 'sell', df)
        else:
            pass
    
    

In [33]:
# log in with the credentials provided in the credentials.ini file
cred = configparser.ConfigParser()
cred.read('config/credentials.ini')
username = cred['credentials']['username'] # ^ this is your username
password = cred['credentials']['password'] # ^ this is your password
percent_to_spend = cred['trader']['percent_to_use'] #^ this is the percent of your account you want to use
stop_loss_percent = cred['trading']['stop_loss_percent'] #^ this is the percent of your account you want to use

print('Logging in...')
r.login(username,password) #^ this logs you in
print('Logged in!')
# now print out the variables
# print('username: ' + username)
# print('password: ' + password)
print('percent_to_spend: ' + percent_to_spend)
print('stop_loss_percent: ' + stop_loss_percent)
# Test the login by printing out the account balance
print('Account balance: ' + str(r.get_crypto_positions()))
df = pd.DataFrame(r.get_crypto_positions())
df.head()


Logging in...
Logged in!
percent_to_spend: 0.8
stop_loss_percent: 0.05
Account balance: [{'account_id': '61f33e4c-8cc5-497e-a296-f0f9691a030e', 'cost_bases': [{'currency_id': '1072fc76-1862-41ab-82c2-485837590762', 'direct_cost_basis': '0.000000000000000000', 'direct_quantity': '0.000000000000000000', 'direct_reward_cost_basis': '0.000000000000000000', 'direct_reward_quantity': '0.000000000000000000', 'direct_transfer_cost_basis': '0.000000000000000000', 'direct_transfer_quantity': '0.000000000000000000', 'id': '6358937c-6d30-46e3-b76b-abc2bf2c7834', 'intraday_cost_basis': '0.000000000000000000', 'intraday_quantity': '0.000000000000000000', 'marked_cost_basis': '0.000000000000000000', 'marked_quantity': '0.000000000000000000'}], 'created_at': '2022-10-25T21:55:08.627884-04:00', 'currency': {'brand_color': '', 'code': 'ADA', 'crypto_type': 'BASE_ASSET', 'display_only': True, 'id': 'f5cd90c4-6058-4e2c-909d-2fe46bb80caf', 'increment': '0.000001000000000000', 'name': 'Cardano', 'type': 'cr

,account_id,cost_bases,created_at,currency,currency_pair_id,id,quantity,quantity_available,quantity_held,quantity_held_for_buy,quantity_held_for_sell,quantity_staked,quantity_transferable,updated_at
0,61f33e4c-8cc5-497e-a296-f0f9691a030e,[{'currency_id': '1072fc76-1862-41ab-82c2-4858...,2022-10-25T21:55:08.627884-04:00,"{'brand_color': '', 'code': 'ADA', 'crypto_typ...",6f2afc32-702e-41ad-b232-ed26b77c1943,6358937c-3759-47f5-b3f1-32d74fae1ca3,0.000000000000000000,0.000000000000000000,0.000000000000000000,0.000000000000000000,0.000000000000000000,0.000000000000000000,0.000000000000000000,2023-06-16T10:31:30.931729-04:00
1,61f33e4c-8cc5-497e-a296-f0f9691a030e,[{'currency_id': '1072fc76-1862-41ab-82c2-4858...,2022-10-25T21:54:46.600396-04:00,"{'brand_color': '', 'code': 'AAVE', 'crypto_ty...",f649e71a-543b-4fd8-9faa-bc6c2c9e3d21,63589366-5dd7-4ea5-8e2d-82c631c26725,0.014000000000000000,0.014000000000000000,0.000000000000000000,0.000000000000000000,0.000000000000000000,0.000000000000000000,0.014000000000000000,2023-07-30T21:11:18.623354-04:00
2,61f33e4c-8cc5-497e-a296-f0f9691a030e,[{'currency_id': '1072fc76-1862-41ab-82c2-4858...,2022-10-25T21:54:34.357790-04:00,"{'brand_color': '', 'code': 'XTZ', 'crypto_typ...",022bb5c2-d394-4e1c-a8c0-5b16ce1eab8d,6358935a-9f05-45ab-a22b-cf1381e48807,1.190000000000000000,1.190000000000000000,0.000000000000000000,0.000000000000000000,0.000000000000000000,0.000000000000000000,1.190000000000000000,2023-07-30T20:36:12.621017-04:00
3,61f33e4c-8cc5-497e-a296-f0f9691a030e,[{'currency_id': '1072fc76-1862-41ab-82c2-4858...,2022-08-08T20:52:59.036795-04:00,"{'brand_color': '', 'code': 'XLM', 'crypto_typ...",7a04fe7a-e3a8-4a07-8c35-d0fec9f35569,62f1afeb-a2c2-40d6-b20d-c8dee94998f6,0.000000000000000000,0.000000000000000000,0.000000000000000000,0.000000000000000000,0.000000000000000000,0.000000000000000000,0.000000000000000000,2023-07-30T15:53:13.347710-04:00
4,61f33e4c-8cc5-497e-a296-f0f9691a030e,[{'currency_id': '1072fc76-1862-41ab-82c2-4858...,2022-08-08T20:52:46.775291-04:00,"{'brand_color': '', 'code': 'AVAX', 'crypto_ty...",f62a25b6-31c0-4c92-846d-3c8105627b62,62f1afde-e0af-4022-b7fa-6a5b0271a568,0.000000000000000000,0.000000000000000000,0.000000000000000000,0.000000000000000000,0.000000000000000000,0.000000000000000000,0.000000000000000000,2023-07-30T20:33:27.511916-04:00


In [43]:
def update_master_order_history(order_history, new_order):
    order_history.append(new_order)
    return order_history

In [50]:
import pandas as pd
from tqdm import tqdm

def calculate_ta_indicators(coins):
    """
    The calculate_ta_indicators function calculates different technical indicators and generates trading signals based on these indicators.
    The indicators are: EMA, MACD, RSI, Williams %R, Stochastic Oscillator, Bollinger Bands, and Parabolic SAR.
    A boolean is generated based on these indicators. If the boolean is True, a buy signal is generated. If the boolean is False, a sell signal is generated.
    The signals are returned in a DataFrame.
    :param coins: A list of coins to generate signals for
    :return: A DataFrame with the trading signals for each coin
    """
    try:
        signals_df = pd.DataFrame()
        bought_coins = set()
        crypto_positions = []  # list of coins in the portfolio
        buying_power = 0  # available funds for buying crypto

        for coin in tqdm(coins):
            if coin not in crypto_positions:
                print(Fore.RED + f'Coin is not in crypto positions list {coin} ...' + Style.RESET_ALL) # print out the coin that is not in the portfolio
            
            if coin not in bought_coins:
            
                # buy the coin if the buy signal is greater than the sell signal
                spend_amount = max(buying_power * float(
                    percent_to_spend), 1.00) # pull from config file

                side = 'buy'
                amount_in = 'dollars'
                quantity_or_price = str(spend_amount)

                r.orders.order_crypto(
                    symbol=coin,
                    quantityOrPrice=float(quantity_or_price),
                    amountIn=str(amount_in),
                    side=side,
                    timeInForce='gtc',
                    jsonify=True
                )
                time.sleep(random.randint(1, 5))

                # update the master order history
                update_master_order_history(
                    coin = coin,
                    close_price = df.close.iloc[-1],
                    spend_amount = spend_amount,
                    side = side
                )
                # print out the order that we just placed
                print(f'Order placed for {quantity_or_price} {amount_in} of {coin} at {datetime.now()}')
                buying_power -= spend_amount
                bought_coins.add(coin)
            elif coin in bought_coins:
                # sell the coin if the sell signal is greater than the buy signal and the coin is in the portfolio
                side = 'sell'
                amount_in = 'crypto'
                quantity_or_price = str(quantity)

                r.orders.order_crypto(
                    symbol=coin,
                    quantityOrPrice=float(quantity_or_price),
                    amountIn=str(amount_in),
                    side=side,
                    timeInForce='gtc',
                    jsonify=True
                )
                time.sleep(random.randint(1, 5))

                # update the master order history
                update_master_order_history(
                    coin = coin,
                    close_price = df.close.iloc[-1],
                    spend_amount = spend_amount,
                    side = side
                )
                # print out the order that we just placed
                print(f'Order placed for {quantity_or_price} {amount_in} of {coin} at {datetime.now()}')
                buying_power += float(quantity_or_price)
                bought_coins.remove(coin)
            
            else:
                print(f'No action to take for {coin}')

            # Update the signals DataFrame
            signals_df = signals_df.append(df)

        print(f'Buying power: {buying_power}')

    except Exception as e:
        print(f'Error: {e}')
        pass

    return signals_df

def update_master_order_history(coin, close_price, spend_amount, side):
    """
    The update_master_order_history function updates the master order history with the latest order.
    :param coin: The coin that was bought or sold
    :param close_price: The closing price of the coin
    :param spend_amount: The amount spent on the coin
    :param side: The side of the order (buy or sell)
    :return: None
    """
    try:
        # Get the master order history
        master_order_history = pd.read_csv('master_order_history.csv', index_col=0)

        # Create a new DataFrame to hold the order data
        order_data = pd.DataFrame()

        # Add the order data to the DataFrame
        order_data['coin'] = [coin]
        order_data['close_price'] = [close_price]
        order_data['spend_amount'] = [spend_amount]
        order_data['side'] = [side]
        order_data['date'] = [datetime.now()]

        # Append the order data to the master order history
        master_order_history = master_order_history.append(order_data)

        # Save the master order history
        master_order_history.to_csv('master_order_history.csv')

    except Exception as e:
        print(f'Error: {e}')
        pass

def get_coins():
    # Get the coins to trade
    coins = ['BTC', 'ETH', 'XRP', 'LTC', 'BCH', 'EOS', 'BNB', 'BSV', 'XLM', 'ADA']
    return coins
def main():
    """
    The main function executes the program.
    :return: None
    """
    try:
        # Get the coins to trade
        coins = get_coins()

        # Calculate the technical indicators and generate the trading signals
        signals_df = calculate_ta_indicators(coins)

        # Save the signals DataFrame
        signals_df.to_csv('signals.csv')

    except Exception as e:
        print(f'Error: {e}')
        pass


In [51]:
main()

  0%|          | 0/10 [00:00<?, ?it/s]

Coin is not in crypto positions list BTC ...
Error: 'DataFrame' object has no attribute 'close'
